In [1]:
# import findspark
# findspark.init()

# import pyspark
# from pyspark.sql import SparkSession

In [2]:
import pandas as pd

In [3]:
# spark = SparkSession.builder \
# .master('local') \
# .appName('read_sensor_data') \
# .config('spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2') \
# .getOrCreate()

In [4]:
sensor1_file = 'data_engineer_test/sensor_1.xlsx'
sensor2_file = 'data_engineer_test/sensor_2.csv'
sensor4_file = 'data_engineer_test/sensor_4.parquet'
sensor4_dd_file = 'data_engineer_test/sensor_4_diff_date.pickle'
sensor5_file = 'data_engineer_test/sensor_5.json'

In [5]:
# sensor 1 - xlsx
s1_pd = pd.read_excel(sensor1_file)
s1_pd['created_timestamp'] = pd.to_datetime(s1_pd['created_timestamp'])


print(s1_pd.shape)
print(min(s1_pd['created_timestamp']), 'to', max(s1_pd['created_timestamp']))
s1_pd.head()

# # s1 pyspark
# s1_sp = spark.createDataFrame(s1_pd)

(6530, 4)
2021-07-05 00:00:24.987000+00:00 to 2021-07-05 23:59:59.594000+00:00


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-07-05 00:00:24.987000+00:00,sens_1,31,Good
1,2021-07-05 00:00:27.833000+00:00,sens_1,29,Good
2,2021-07-05 00:01:17.633000+00:00,sens_1,31,Good
3,2021-07-05 00:01:25.107000+00:00,sens_1,29,Good
4,2021-07-05 00:04:40.549000+00:00,sens_1,31,Bad


In [6]:
# sensor 2 - csv
s2_pd = pd.read_csv(sensor2_file)
s2_pd['created_timestamp'] = pd.to_datetime(s2_pd['created_timestamp'])

print(s2_pd.shape)
print(min(s2_pd['created_timestamp']), 'to', max(s2_pd['created_timestamp']))
s2_pd.head()

(7076, 4)
2021-07-05 00:00:07.215000+00:00 to 2021-07-05 23:59:58.274000+00:00


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-07-05 00:00:07.215000+00:00,sens_2,58,Good
1,2021-07-05 00:00:27.833000+00:00,sens_2,57,Good
2,2021-07-05 00:00:30.792000+00:00,sens_2,58,Good
3,2021-07-05 00:00:45.338000+00:00,sens_2,57,Good
4,2021-07-05 00:00:48.289000+00:00,sens_2,58,Good


In [7]:
# sensor 4 - parquet
s4_pd = pd.read_parquet(sensor4_file)
s4_pd['created_timestamp'] = pd.to_datetime(s4_pd['created_timestamp'])

print(s4_pd.shape)
print(min(s4_pd['created_timestamp']), 'to', max(s4_pd['created_timestamp']))
s4_pd.head()

(715, 4)
2021-08-15 00:01:05.292000+00:00 to 2021-08-15 02:30:42.913000+00:00


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-08-15 00:01:05.292000+00:00,sens_4,12.0,Good
1,2021-08-15 00:01:15.689000+00:00,sens_4,0.0,Good
2,2021-08-15 00:05:13.145000+00:00,sens_4,2.0,Good
3,2021-08-15 00:05:17.191000+00:00,sens_4,0.0,Good
4,2021-08-15 00:05:24.189000+00:00,sens_4,3.0,Good


In [8]:
# sensor 4 diff date - pickle
s4_dd_pd = pd.read_pickle(sensor4_dd_file)
s4_dd_pd['created_timestamp'] = pd.to_datetime(s4_dd_pd['created_timestamp'])

print(s4_dd_pd.shape)
print(min(s4_dd_pd['created_timestamp']), 'to', max(s4_dd_pd['created_timestamp']))
s4_dd_pd.head()

(4551, 4)
2021-07-10 07:07:22.871000+00:00 to 2021-07-10 22:42:32.892000+00:00


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-07-10 07:07:22.871000+00:00,sens_4,12.0,Good
1,2021-07-10 07:07:28.903000+00:00,sens_4,0.0,Good
2,2021-07-10 07:08:04.135000+00:00,sens_4,1.0,Good
3,2021-07-10 07:08:08.182000+00:00,sens_4,7.0,Good
4,2021-07-10 07:08:11.042000+00:00,sens_4,11.0,Good


In [10]:
# sensor 5 - json
s5_pd = pd.read_json(sensor5_file, orient='index')
s5_pd['created_timestamp'] = pd.to_datetime(s5_pd['created_timestamp'])


print(s5_pd.shape)
print(min(s5_pd['created_timestamp']), 'to', max(s5_pd['created_timestamp']))
s5_pd.head()

(7185, 4)
2021-06-08 00:00:20.201000+00:00 to 2021-06-08 23:59:48.766000+00:00


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-06-08 00:00:20.201000+00:00,sens_5,45.0,Good
1,2021-06-08 00:00:24.469000+00:00,sens_5,46.0,Good
2,2021-06-08 00:01:05.046000+00:00,sens_5,45.0,Good
3,2021-06-08 00:01:19.484000+00:00,sens_5,46.0,Good
4,2021-06-08 00:01:26.833000+00:00,sens_5,45.0,Good


In [13]:
df = pd.concat([s1_pd, s2_pd, s4_pd, s4_dd_pd, s5_pd])

print(df.shape)
df.head()

(26057, 4)


,created_timestamp,tag_key,tag_val,tag_quality
0,2021-07-05 00:00:24.987000+00:00,sens_1,31.0,Good
1,2021-07-05 00:00:27.833000+00:00,sens_1,29.0,Good
2,2021-07-05 00:01:17.633000+00:00,sens_1,31.0,Good
3,2021-07-05 00:01:25.107000+00:00,sens_1,29.0,Good
4,2021-07-05 00:04:40.549000+00:00,sens_1,31.0,Bad


In [21]:
df_good = df[df['tag_quality']=='Good']

In [22]:
pd.pivot_table(df_good, values='tag_val', index='created_timestamp', columns='tag_key')

tag_key,sens_1,sens_2,sens_4,sens_5
created_timestamp,,,,
2021-06-08 00:00:20.201000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:00:24.469000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:05.046000+00:00,NaN,NaN,NaN,45.0
2021-06-08 00:01:19.484000+00:00,NaN,NaN,NaN,46.0
2021-06-08 00:01:26.833000+00:00,NaN,NaN,NaN,45.0
...,...,...,...,...
2021-08-15 02:23:47.042000+00:00,NaN,NaN,0.0,NaN
2021-08-15 02:23:49.790000+00:00,NaN,NaN,1.0,NaN
2021-08-15 02:23:53.729000+00:00,NaN,NaN,0.0,NaN


In [20]:
df[df['created_timestamp']==pd.to_datetime('2021-08-15 02:30:42.913000+00:00')]

,created_timestamp,tag_key,tag_val,tag_quality
714,2021-08-15 02:30:42.913000+00:00,sens_4,0.0,Good
